# Data Mining for Business Analytics

## Similarity, Neighbors

Spring 2019 - Prof. George Valkanas

Material based on content courtesy of Prof. Foster Provost

***


In [ ]:
# Import the libraries we will be using
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import train_test_split

from scipy.spatial.distance import pdist, squareform
from scipy.cluster.hierarchy import linkage, dendrogram

from sklearn.cluster import KMeans
from sklearn.neighbors import KNeighborsClassifier
from scipy.spatial import distance

%matplotlib inline
import matplotlib.pylab as plt
plt.rcParams['figure.figsize'] = 10, 8



## Example 1: Wine Recommendation

**Customer:** I'd like to order the _Cabernet Sauvignon_ 

**Waiter:** We're just out of this wine.

**Customer:** What would be _the closest_ wine that you currently have?

**Waiter:** We have some very nice _Merlot_ or you could try the _Malbec-Cabernet_


<br/>

#### <div style="color: red">How did the waiter know about this recommendation?</div>

Did she/he know that the customer _will like_ the suggestion? How would _you_ make such a recommendation ?


## Example 2: Plagiarism Detection / Near-Duplicate Detection in Web Search


Plagiarism is not just an unethical practice, but can also have legal ramifications. Plagiarism arises when someone (a person, an entity) copies someone else's (literary) work (without the proper attribution) and tries to pass it as their own.

<br/>

<div style="text-align: right"><i>"When you steal from one author, it's plagiarism; if you steal from many, it's research."<br/> Wilson Mizner</i>
</div>


Meanwhile, when you search the web with your favorite search engine, you'd like to get back results, each of which conveys _new_ information. That is, if all of the first 10 results **all** present **the same** information, you'd not be very happy about it. The first link will have new information (and answer your question), the second one may just reinforce what you learned from the 1st link, but after that it's just more of the same.


Notice that these two problems are effetively the same. The way that the results are used is different in the two scenarios.

<br/>

#### <div style="color: red">How can we solve this problem?</div>

***


## Example 3: Player Substitution

Imagine that you're playing your favorite video game (football, basketball, etc) and one of the players gets injured and needs to be taken off the court / field. 

Which player do you replace them with?

***


#### Fundamental Question

We have seen various classifiers until now. The main question to answer here is the following:

**<div style="text-align: center">Do we have a target variable that we can use?</div>** 

-

-

-

-

-

-

-

### _Show me your friends..._

Unlike magnets, _similars_ attract when it comes to people. That is, our friends' interests are more aligned with our own. Another way to see this is that we may tend to befriend individuals who have _common interests_ with our own.

In abstract terms, we are _more similar_ with people we are friends with and _less similar_ with people who we (knowingly / decidedly) aren't friends with.


***


### What are you talking about?

Similarity between **people**? Between **wines**? Between **texts**? What are you talking about? How can these things be possibly related?

#### Let's start simple

- Can you determine the similarity between two integers? For example, which two numbers would you say are _more similar_? Numbers 9 and 10,  or the numbers 1 and 100?
    * What about negative numbers? How about -3 and -4 compared against -100 and -98?


- Can you come up with a similarity measure for two real-valued numbers? Is it any different from integer values?


- Can you measure the similarity between GPS signals? A GPS signal is a pair of (x, y) coordinates, commonly refered to as _latitude_ and _longitude_. Let's say that you are given 3 pairs of such GPS coordinates:
    * **GPS 1:** (1, 1)
    * **GPS 2:** (1, 0.5)
    * **GPS 3:** (2, 2)
    
  Which pair of GPS coordinates is _more similar_ ? How did you compute the similarity?


- OK, let's go for something more involved. Imagine that you are given 3 baskets with fruits:
    * **Basket 1:** Fruit_1, Fruit_2, Fruit_3
    * **Basket 2:** Fruit_1, Fruit_2, Fruit_4, Fruit_5
    * **Basket 3:** Fruit_3, Fruit_2
    
  What if our basket had multiples of the same fruit, e.g., **Bakset 1:** Fruit_1, Fruit_1, Fruit_2, Fruit_3 ?
    
***

### Back to the difficult cases

How do you define the similarity between:

- Two _wines_?
- Two _pieces of text_ ?
- Two _people_ ?

Generalizing from our previous examples, we are discussing about the similarity of _objects_. 

**Question:** Did we use just a single similarity?

In case it's also not obvious by now, **to compute the similarity between _objects_ we need to have quantifiable _features_**.

## Enough talk! Let's work with some Data

Following that discussion, we have compiled a scotch whiskey data set. You can find it in `data/scotch.csv`.

The data consists of 5 general whiskey attributes, each of which has many possible values:

- **Color**: yellow, very pale, pale, pale gold, gold, old gold, full gold, amber, etc.
- **Nose**: aromatic, peaty, sweet, light, fresh, dry, grassy, etc.
- **Body**: soft, medium, full, round, smooth, light, firm, oily.
- **Palate**: full, dry, sherry, big, fruity, grassy, smoky, salty, etc.
- **Finish**: full, dry, warm, light, smooth, clean, fruity, grassy, smoky, etc.

Let's read the file in and take a look. For convenience, we have also dummysized the data. There are a few other features unrelated to the ones above. For this class, we will be dropping them. However, feel free to check them out!



In [ ]:
data = pd.read_csv("data/scotch.csv")

data = data.drop([u'age', u'dist', u'score', u'percent', u'region', u'district', u'islay', u'midland', u'spey', u'east', u'west', u'north ', u'lowland', u'campbell', u'islands'], axis=1)

In [ ]:
data.head()


We've discussed earlier that there are many similarity measures.  Similarity is often cast as "closeness" in some space, computed by a distance measure.  Often in data science, the terms similarity and distance are used interchangeably (a little strangely to the uninitiated). 

We'll use the library scipy.spatial.distance available [here](http://docs.scipy.org/doc/scipy/reference/spatial.distance.html)

This library has functions to compute the distance between two **numeric** vectors. In particular, **pdist(X[, metric, p, w, V, VI])**	computes pairwise distances between the observations in n-dimensional space.

The _metric_ parameter refers to the **distance function** used to compute the distance between the instances (observations) and can be:
* ‘braycurtis’, 
* ‘canberra’, 
* ‘chebyshev’, 
* ‘cityblock’, 
* ‘correlation’, 
* ‘cosine’, 
* ‘dice’, 
* ‘euclidean’, 
* ‘hamming’, 
* ‘jaccard’, 
* ‘kulsinski’, 
* ‘mahalanobis’, 
* ‘matching’, 
* ‘minkowski’, 
* ‘rogerstanimoto’, 
* ‘russellrao’, 
* ‘seuclidean’, 
* ‘sokalmichener’, 
* ‘sokalsneath’, 
* ‘sqeuclidean’, 
* ‘yule’

Before diving in deep, let's see how the method actually works

In [ ]:
# Euclidean Distance
sample_data = [ [1,2,3], [4,5,6], [7,8,9] ]
distance.pdist(sample_data, 'euclidean')

Well, having all of that in the same row isn't particularly helpful. What are we really looking at here?
Let's better format our output.

In [ ]:
# Euclidean Distance
sample_data = [ [1,2,3], [4,5,6], [7,8,9] ]
distance.squareform(distance.pdist(sample_data, 'euclidean'))

OK, that's better. Let's do the same for the other distances.

In [ ]:
# What about Jaccard distance
sample_data = [ [0,0,1,1,1], [0,0,1,0,1], [1,0,0,0,1] ]
distance.squareform(distance.pdist(sample_data, 'jaccard'))

In [ ]:
# What about Hamming distance
sample_data = [ [0,0,1,1,1], [0,0,1,0,1], [1,0,0,0,1] ]
distance.squareform(distance.pdist(sample_data, 'hamming'))


Alright, now that we got the hang of it, let's make our life easier as follows.  Here is a function that will compute a number of distances of interest that we will provide ( $distance_measures$ parameter ).

In [ ]:

def whiskey_distance(whiskey_name, distance_measures, k):
    
    """
    whiskey_name: The name of the whiskey to search for
    distance_measures: A list containing the different distances that we want to compute
    k: The number of similar objects that we want to retrieve
    """
    
    # We want a data frame to store the output
    # distance_measures is a list of the distance measures you want to compute (see below)
    # n is how many "most similar" to report
    distances = pd.DataFrame()
    
    # Find the location of the whiskey we are looking for
    whiskey_location = np.where(data.index == whiskey_name)[0][0]

    # Go through all distance measures we care about
    for distance_measure in distance_measures:
        # Find all pairwise distances
        current_distances = distance.squareform(distance.pdist(data, distance_measure))
        # Get the closest n elements for the whiskey we care about
        most_similar = np.argsort(current_distances[:, whiskey_location])[0:k]
        # Append results (a new column to the dataframe with the name of the measure)
        distances[distance_measure] = list(zip(data.index[most_similar], current_distances[most_similar, whiskey_location]))
    return distances


In [ ]:

# Let's test the distance of one of the whiskeys that are in there
whiskey_distance('Bunnahabhain', ['euclidean'], 6)

What about more distances? What is the output then?

In [ ]:
#Let's try more distances for the same whiskey
whiskey_distance('Bunnahabhain', ['euclidean', 'cityblock', 'cosine', 'jaccard'], 10)

## Show me your friends... and I'll make a prediction

Is that the only thing we can do with similarity? No, not really.  We can also use similarity between objects (i.e., instances) to make _predictions_. To do so, we need to have labels on the _similar_ instances. How do you think we would do that?

See the image below to guide your thinking. We want to classify the green instance. What should its class be?

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/e/e7/KnnClassification.svg/850px-KnnClassification.svg.png" width="30%" />

[Source: Wikipedia](https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm#/media/File:KnnClassification.svg)

***


The above approach is known as a **$k$-Nearest Neighbor** (**kNN**) classifier, because it finds the $k$ _most similar_ neighbors of the _instance_ that we want to predict.

In the example of the Figure above, we used the Euclidean distance between two items (because... circles!). We collected the $k$ most similar instances in our datasets and combined their target variables to make our prediction.

**Question:** How did we combine the target variables of the neighbors?

**Question:** How would we combine the target variable for a _regression_ problem?

**Question:** **kNN** is a _lazy_ classifier. Can you think why we call it that?

**Question:** What happens if we set $k$ to a _very large_ value, e.g., the total number of instances in our dataset?

Let's work with our mailing campaign dataset which has a target variable.

In [ ]:
mail_data = pd.read_csv("data/mailing.csv")
mail_data = pd.get_dummies(mail_data)
mail_data.head()

In [ ]:
# The "data" dataframe contains everything together.
# Get the features separately from the class.
X = mail_data.drop(['class'], axis=1)
Y = mail_data['class']

In [ ]:
# Keep 75% of the data for training and 25% for testing. We have used this method before.
X_mailing_train, X_mailing_test, Y_mailing_train, Y_mailing_test = train_test_split(X, Y, train_size=.75)

***
Great! We've read the data in, let's now learn a **kNN** classifier on our data.  The main parameter to consider here is the $k$ value, i.e., the number of neighbors that we want to base our decision making on.  For a full description of the parameters that the **kNN** classifier takes is available [here](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html).

For example, check what happens when the **weights** parameter isn't specified, versus the parameter being set to 'distance', i.e., **weights='distance'**.

Also, check out the parameter **p** which controls the power of the Minkowski distance.

The **metric** parameter controls which distance measure to consider between instances. More details about the values that the **metric** parameter can take can be found [here](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.DistanceMetric.html).

In [ ]:
# Let's train a k-Nearest Neighbor classifier
model_mailing = KNeighborsClassifier(10, weights='distance')   # Also try it with weights='distance'
model_mailing.fit(X_mailing_train, Y_mailing_train)


In [ ]:
# Apply the trained model on the testing dataset and get the predicted classes
predictions = model_mailing.predict(X_mailing_test)

# Let's generate a confusion matrix
conf_mtx = metrics.confusion_matrix(Y_mailing_test, predictions, labels=[1, 0])

# Let's turn the confusion matrix into a DataFrame, to make it more presentable
conf_mtx_df = pd.DataFrame(conf_mtx.T, columns=['(True) p', '(True) n'], index=['[Predicted] Y', '[Predicted] N'])
conf_mtx_df

### Try it out with some different $k$-values

In [ ]:

for k in [1, 10, 50, 100, 1000, 2000]:
    model = KNeighborsClassifier(k, weights='distance')
    model.fit(X_mailing_train, Y_mailing_train)
    print ("Accuracy with k = %d is %.3f" % (k, metrics.accuracy_score(Y_mailing_test, model.predict(X_mailing_test))) )
    

### Question for you

- Advantages of kNN ?
- Disadvantages of kNN ?
- Also check the previous questions in this notebook.